In [1]:
from functions import read_data
import pandas as pd
import os 
print(os.getcwd())

/Users/joaoprado/ada-2022-project-hotchocolate


In [2]:
BA_dataset = pd.read_csv("DATA/BeerAdvocate_ratings_part_0.csv")
BA_breweries = read_data.fetch_csv("DATA/BeerAdvocate.tar","breweries")
RB_dataset = pd.read_csv("DATA/RateBeer_ratings_part_0.csv")
RB_breweries = read_data.fetch_csv("DATA/RateBeer.tar","breweries")
display(RB_dataset)

,Unnamed: 0,beer_name,beer_id,brewery_name,brewery_id,style,abv,date,username,user_id,appearance,aroma,palate,taste,overall,rating,text
0,0,33 Export (Gabon),410549,Sobraga,3198,Pale Lager,5.0,1461664800,Manslow,175852,2,4,2,4,8,2.0,"Puszka 0,33l dzięki Christoph . Kolor jasnozł..."
1,1,Castel Beer (Gabon),105273,Sobraga,3198,Pale Lager,5.2,1487329200,MAGICuenca91,442761,2,3,2,4,8,1.9,Cerveza pale lager gabonesa. MÃ¡s floja que l...
2,2,Castel Beer (Gabon),105273,Sobraga,3198,Pale Lager,5.2,1466762400,Sibarh,288889,3,3,2,3,5,1.6,"Kolor- złoty, klarowny. Piana - drobna, średn..."
3,3,Castel Beer (Gabon),105273,Sobraga,3198,Pale Lager,5.2,1451646000,fombe89,250510,4,3,1,2,5,1.5,"Botella, de GabÃ³n regalo familiar.31/01/2015..."
4,4,Castel Beer (Gabon),105273,Sobraga,3198,Pale Lager,5.2,1445594400,kevnic2008,122778,2,4,2,4,7,1.9,Many thanks for this beer to Erzengel. Pours ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499995,499995,Bad Guy Anastasia,219990,Bad Guy Brewery,17089,American Pale Ale,6.0,1392548400,Dorqui,17425,4,7,4,7,13,3.5,"Draught at ""GolosItalia 2014"". Reddish amber ..."
499996,499996,Bad Guy Anastasia,219990,Bad Guy Brewery,17089,American Pale Ale,6.0,1393844400,daniele,55098,3,7,4,6,13,3.3,ambrata buona schiuma piuttosto persistente n...
499997,499997,Bad Guy Foxy Lady,219986,Bad Guy Brewery,17089,California Common,5.5,1396519200,sebletitje,74926,3,7,3,7,15,3.5,Draft @ Fiera della birra di Forli â14. Cui...
499998,499998,Bad Guy Foxy Lady,219986,Bad Guy Brewery,17089,California Common,5.5,1393498800,Silvan7,201189,4,8,3,7,14,3.6,"On tap, golositalia 2014. Rara California Com..."


In [3]:
BA_df = BA_dataset.merge(BA_breweries[["id","location"]], how="left",left_on= "brewery_id",right_on="id")
beer_most_drinked_by_country_BA =BA_df[["location","beer_name","id","rating"]].groupby(by=["location","beer_name"]).agg({"id":"count","rating":"sum"}).reset_index()
style_most_drinked_by_country_BA =BA_df[["location","style","id","rating"]].groupby(by=["location","style"]).agg({"id":"count","rating":"sum"}).reset_index()

for number in range(1,17):
    temp_BA = pd.read_csv(f"DATA/BeerAdvocate_ratings_part_{number}.csv")
    temp_BA = temp_BA.merge(BA_breweries[["id","location"]], how="left",left_on= "brewery_id",right_on="id")

    temp_grouped_on_beer_BA = temp_BA[["location","beer_name","id","rating"]].groupby(by=["location","beer_name"]).agg({"id":"count","rating":"sum"}).reset_index()
    temp_grouped_on_style_BA = temp_BA[["location","style","id","rating"]].groupby(by=["location","style"]).agg({"id":"count","rating":"sum"}).reset_index()


    beer_most_drinked_by_country_BA = pd.concat([beer_most_drinked_by_country_BA, temp_grouped_on_beer_BA]).groupby(['location', 'beer_name']).sum().reset_index()
    style_most_drinked_by_country_BA =  pd.concat([style_most_drinked_by_country_BA, temp_grouped_on_style_BA]).groupby(['location', 'style']).sum().reset_index()

RB_df = RB_dataset.merge(RB_breweries[["id","location"]], how="left",left_on= "brewery_id",right_on="id")
beer_most_drinked_by_country_RB =RB_df[["location","beer_name","id","rating"]].groupby(by=["location","beer_name"]).agg({"id":"count","rating":"sum"}).reset_index()
style_most_drinked_by_country_RB =RB_df[["location","style","id","rating"]].groupby(by=["location","style"]).agg({"id":"count","rating":"sum"}).reset_index()

for number in range(1,15):
    temp_RB = pd.read_csv(f"DATA/RateBeer_ratings_part_{number}.csv")
    temp_RB = temp_RB.merge(RB_breweries[["id","location"]], how="left",left_on= "brewery_id",right_on="id")
    temp_grouped_on_beer_RB = temp_RB[["location","beer_name","id","rating"]].groupby(by=["location","beer_name"]).agg({"id":"count","rating":"sum"}).reset_index()
    temp_grouped_on_style_RB = temp_RB[["location","style","id","rating"]].groupby(by=["location","style"]).agg({"id":"count","rating":"sum"}).reset_index()
    beer_most_drinked_by_country_RB = pd.concat([beer_most_drinked_by_country_RB, temp_grouped_on_beer_RB]).groupby(['location', 'beer_name']).sum().reset_index()
    style_most_drinked_by_country_RB = pd.concat([style_most_drinked_by_country_RB, temp_grouped_on_style_RB]).groupby(['location', 'style']).sum().reset_index()

/var/folders/76/nc56p_fj6gq8l_bh8nsg44kr0000gn/T/ipykernel_42892/2825390250.py:21: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  temp_RB = pd.read_csv(f"DATA/RateBeer_ratings_part_{number}.csv")
/var/folders/76/nc56p_fj6gq8l_bh8nsg44kr0000gn/T/ipykernel_42892/2825390250.py:21: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  temp_RB = pd.read_csv(f"DATA/RateBeer_ratings_part_{number}.csv")
/var/folders/76/nc56p_fj6gq8l_bh8nsg44kr0000gn/T/ipykernel_42892/2825390250.py:21: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  temp_RB = pd.read_csv(f"DATA/RateBeer_ratings_part_{number}.csv")


In [4]:
#We renormalize the averages that were aggregated from all parts of the dataset
beer_most_drinked_by_country_BA["normalized_rating"] = beer_most_drinked_by_country_BA.apply(lambda row: row["rating"]/row["id"],axis=1)
style_most_drinked_by_country_BA["normalized_rating"] = style_most_drinked_by_country_BA.apply(lambda row: row["rating"]/row["id"],axis=1)

beer_most_drinked_by_country_RB["normalized_rating"] = beer_most_drinked_by_country_RB.apply(lambda row: row["rating"]/row["id"],axis=1)
style_most_drinked_by_country_RB["normalized_rating"] = style_most_drinked_by_country_RB.apply(lambda row: row["rating"]/row["id"],axis=1)

#Sanity check : 
# For BeerAdvocate, we have 16 csvs with 500'000 ratings and 1 csv with 393'032 ratings. 
#We should have 8'393'032 in total.
print(beer_most_drinked_by_country_BA["id"].sum())
#Sanity check : 
# For Ratebeer, we have 14 csvs with 500'000 ratings and 1 csv with 122'074 ratings. 
#We should have 7'122'074 in total.
print(beer_most_drinked_by_country_RB["id"].sum())


8393032
7122074


In [5]:
def most_rated(df):
    df = df.reset_index()
    row = df.iloc[df['id'].idxmax()].copy()
    return row
def best_rating(df,threshold,metric):
    df = df.reset_index()
    row = df.loc[[0]].copy()
    df = df[df["id"] > threshold].copy()
    if len(df) == 0:
        row[metric] = "No beer with enough votes"
        return row
    else : 
        row = df.loc[[df['rating'].idxmax()]].copy()
        return row
most_reviewed_beer_BA = beer_most_drinked_by_country_BA.groupby(by="location").apply(lambda df : most_rated(df)).reset_index(drop=True)
most_reviewed_beer_RB = beer_most_drinked_by_country_RB.groupby(by="location").apply(lambda df : most_rated(df)).reset_index(drop=True)
favorite_beer_BA = beer_most_drinked_by_country_BA.groupby(by="location").apply(lambda df : best_rating(df,10,"beer_name")).reset_index(drop=True)
favorite_beer_RB = beer_most_drinked_by_country_RB.groupby(by="location").apply(lambda df : best_rating(df,10,"beer_name")).reset_index(drop=True)

most_reviewed_style_BA = style_most_drinked_by_country_BA.groupby(by="location").apply(lambda df : most_rated(df)).reset_index(drop=True)
most_reviewed_style_RB = style_most_drinked_by_country_RB.groupby(by="location").apply(lambda df : most_rated(df)).reset_index(drop=True)
favorite_style_BA = style_most_drinked_by_country_BA.groupby(by="location").apply(lambda df : best_rating(df,10,"style")).reset_index(drop=True)
favorite_style_RB = style_most_drinked_by_country_RB.groupby(by="location").apply(lambda df : best_rating(df,10,"style")).reset_index(drop=True)

In [6]:
#Save work
display(most_reviewed_style_RB)
most_reviewed_beer_BA.to_csv("DATA/most_reviewed_beer_BA.csv",index=True)
most_reviewed_beer_RB.to_csv("DATA/most_reviewed_beer_RB.csv",index=True)
most_reviewed_style_BA.to_csv("DATA/most_reviewed_style_BA.csv",index=True)
most_reviewed_style_RB.to_csv("DATA/most_reviewed_style_RB.csv",index=True)
favorite_beer_RB.to_csv("DATA/favourite_beer_RB.csv",index=True)
favorite_beer_BA.to_csv("DATA/favourite_beer_BA.csv",index=True)
favorite_style_RB.to_csv("DATA/favourite_style_RB.csv",index=True)
favorite_style_BA.to_csv("DATA/favourite_style_BA.csv",index=True)

,index,location,style,id,rating,normalized_rating
0,2,Abkhazia,Pale Lager,21,37.4,1.780952
1,18,Albania,Pale Lager,419,802.0,1.914081
2,23,Algeria,Pale Lager,56,97.5,1.741071
3,30,Andorra,Golden Ale/Blond Ale,46,108.1,2.350000
4,41,Angola,Pale Lager,97,175.0,1.804124
...,...,...,...,...,...,...
260,8569,Virgin Islands (British),Premium Bitter/ESB,1,3.5,3.500000
261,8576,Virgin Islands (U.S.),India Pale Ale (IPA),191,580.3,3.038220
262,8592,Wales,Bitter,4540,13580.3,2.991256
263,8641,Zambia,Pale Lager,57,115.5,2.026316
